In [3]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [96]:
generate_item_dict(1, [1,2,3,4,5,6,7,8,9])

7
{'order_id': 1, 'item_id': 7, 'description': "Description of item ['fbbIKgf2gQ', 'PYM7jfIROe', 'w74rjrSet7']"}


{'order_id': 1,
 'item_id': 7,
 'description': "Description of item ['fbbIKgf2gQ', 'PYM7jfIROe', 'w74rjrSet7']"}

In [94]:
from secrets import choice 
from string import ascii_letters, digits
from scipy.stats import poisson 
from numpy.random import randint, shuffle
from typing import Union
from numpy.random import randint
from random import sample

DEFAULT_TOKEN_LENGTH = 10

def generate_random_tokens(
    num_tokens: int = 1, token_length: int = DEFAULT_TOKEN_LENGTH
):
    tokens = []
    for _ in range(num_tokens):
        token = ''.join(choice(ascii_letters + digits) for _ in range(token_length))
        tokens.append(token)
    
    return tokens

def generate_item_ids(num_items: int): 
    return generate_random_tokens(num_tokens, DEFAULT_TOKEN_LENGTH)

def generate_item_dict(itemset_id: Union[int, str], item_ids:list):
    # Randomly select an item id for items
    item_id = sample(item_ids, 1)[0]
    
    item_description = f"Description of item {item}"
    item_dict = {
        'order_id': itemset_id, 
        'item_id': item_id, 
        'description': item_description
    }

    print(item_dict)
    
    return item_dict

def generate_itemset_list(itemset_id: int, item_ids:list):
    # Mean number of items per itemsets
    mean_items_per_itemset = 5
    
    # Generate bin sizes using Poisson distribution
    itemset_size = poisson.rvs(mean_items_per_itemset, size=1)[0]

    for item in range(itemset_size):
        item_dict = generate_item_dict(itemset_id, item_ids)
        itemsets.append(item_dict)

    return itemsets

def generate_itemsets_dataframe(num_items: int, num_itemsets):
    # Generate available item ids
    item_ids = generate_item_ids(num_items)
    
    # Generate orders, items, and descriptions
    itemsets = []
    for itemset_id in range(num_itemsets):
        itemset_id_ = itemset_id + 1
    
        itemset = generate_itemset_list(itemset_id_, item_ids)
        itemsets = itemsets + itemset
    
    # Create a DataFrame from the generated data
    return pd.DataFrame(itemsets)

In [95]:
generate_itemset_list(1, [1,2,3])

3
{'order_id': 1, 'item_id': 3, 'description': "Description of item ['fbbIKgf2gQ', 'PYM7jfIROe', 'w74rjrSet7']"}
2
{'order_id': 1, 'item_id': 2, 'description': "Description of item ['fbbIKgf2gQ', 'PYM7jfIROe', 'w74rjrSet7']"}
2
{'order_id': 1, 'item_id': 2, 'description': "Description of item ['fbbIKgf2gQ', 'PYM7jfIROe', 'w74rjrSet7']"}


[{'order_id': 1,
  'item_id': ['gXQOCCUPDk'],
  'description': "Description of item ['gXQOCCUPDk']"},
 {'order_id': 1,
  'item_id': ['gXQOCCUPDk'],
  'description': "Description of item ['gXQOCCUPDk']"},
 {'order_id': 1,
  'item_id': ['2qRZUobRfN'],
  'description': "Description of item ['2qRZUobRfN']"},
 {'order_id': 1,
  'item_id': ['ckywhZk9qy'],
  'description': "Description of item ['ckywhZk9qy']"},
 {'order_id': 1,
  'item_id': ['gXQOCCUPDk'],
  'description': "Description of item ['gXQOCCUPDk']"},
 {'order_id': 1,
  'item_id': ['ckywhZk9qy'],
  'description': "Description of item ['ckywhZk9qy']"},
 {'order_id': 1,
  'item_id': ['2qRZUobRfN'],
  'description': "Description of item ['2qRZUobRfN']"},
 {'order_id': 1,
  'item_id': ['gXQOCCUPDk'],
  'description': "Description of item ['gXQOCCUPDk']"},
 {'order_id': 1,
  'item_id': ['ckywhZk9qy'],
  'description': "Description of item ['ckywhZk9qy']"},
 {'order_id': 2,
  'item_id': ['vmE0tnEzXR'],
  'description': "Description of ite

In [82]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

# Set seed for reproducibility
np.random.seed(0)

# Parameters
num_itemsets = 10
num_items = 10

itemsets_df = generate_itemsets_dataframe(num_items, num_itemsets)

# Display the DataFrame
print(itemsets_df)


{'order_id': 1, 'item_id': 'us7I68uMgl', 'description': 'Description of item us7I68uMgl'}
{'order_id': 1, 'item_id': 'KhMT4DCn2e', 'description': 'Description of item KhMT4DCn2e'}
{'order_id': 1, 'item_id': 'KhMT4DCn2e', 'description': 'Description of item KhMT4DCn2e'}
{'order_id': 1, 'item_id': 'ogBCxBaRMg', 'description': 'Description of item ogBCxBaRMg'}
{'order_id': 1, 'item_id': 'ogBCxBaRMg', 'description': 'Description of item ogBCxBaRMg'}
{'order_id': 1, 'item_id': 'ogBCxBaRMg', 'description': 'Description of item ogBCxBaRMg'}
{'order_id': 1, 'item_id': 'eCUXPRtQTj', 'description': 'Description of item eCUXPRtQTj'}
{'order_id': 1, 'item_id': 'Ov0WCyT4Zx', 'description': 'Description of item Ov0WCyT4Zx'}
{'order_id': 1, 'item_id': 'KhMT4DCn2e', 'description': 'Description of item KhMT4DCn2e'}
{'order_id': 1, 'item_id': 'Ov0WCyT4Zx', 'description': 'Description of item Ov0WCyT4Zx'}
{'order_id': 1, 'item_id': 'Ov0WCyT4Zx', 'description': 'Description of item Ov0WCyT4Zx'}
{'order_id

In [58]:
import numpy as np
np.random.sample([1,2,3])

array([[[0.0202184 , 0.83261985, 0.77815675],
        [0.87001215, 0.97861834, 0.79915856]]])